**<center><span style="font-size:88px;">openrouteservice - route the world dynamically</span></center>**

### <center>Nils Nolde</center>
#### <center>HeiGIT - University of Heidelberg</center>

# Fact Sheet

- **openrouteservice** is a project of the KTS backed HeiGIT institute at the University of Heidelberg
- started in 2008, **openrouteservice** was one of the first routing API's
- entirely **open-source**, backend written in **Java**
- based on **OpenStreetMap** data
- 2500 requests per day **for free**

### Available API's
> **Directions**

> **Isochrones**

> **Geocoding**

> **Matrix**

> **Places**

## Local installation

<span style="font-size: 1.1em; font-weight: bold;">License: </span><a href="https://opensource.org/licenses/MIT">MIT</a>

```bash
git clone https://github.com/GIScience/openrouteservice
cd docker && sudo docker-compose up```

Approx. **64-128 GB RAM** per profile for `planet.pbf`.

<span style="font-size: 1.1em; font-weight: bold;">Geocoding</span> and <span style="font-size: 1.1em; font-weight: bold;">Places</span> need special setup:

<https://github.com/pelias/pelias>

<https://github.com/GIScience/openpoiservice>

# <center style="font-size: 200px">Architecture</center>

![blabla](../img/architecture.png)

In [ ]:
import folium
import branca
from folium.plugins import MeasureControl
from shapely import geometry
from IPython.display import HTML, display, clear_output
from ipywidgets import widgets, interact, interactive
import json
from urllib.parse import urlencode
import requests

from pprint import pprint
from numpy import average

api_key = json.load(open('token.json'))

In [ ]:
# Set up some fundamentals

# Style function for folium GeoJson
def style_function(opacity=1):
    return lambda feature: dict(weight=7,
                                opacity=opacity,
                               )

# Base map factory for interactive examples
def createBaseMap(width, height, coords=(49.406486, 8.683481), zoom=15, map_provider='stamenwatercolor'):
    fig = folium.Figure(width=width, height=height)
    m = folium.Map(tiles=map_provider, location=coords, zoom_start=zoom)
    
    m.add_child(MeasureControl())
#     m.add_child(folium.LayerControl())
    
    return fig, m

## Directions API

Classical routing API, based on the excellent [GraphHopper](https://github.com/graphhopper/graphhopper) stack.

<div style="margin:0px 20px; width: 40%; display: inline-block; vertical-align: top;">
<h3>Features</h3>
<br>
<div style=" border-right: 1px solid black">
<ul>
    <li>10 routing profiles:
        <ul>
            <li>2 car: `driving-car`, `driving-hgv`
            <li>5 cycling, incl. `*-mountain`, `*-safe`
            <li>2 walking: `foot-walking`, `foot-hiking`
            <li><strong>`wheelchair`</strong>
        </ul>
    <li>Output formats: `geojson`, `gpx`
    <li>13 languages for instructions
    <li>Returns extra way information for route segments, e.g. steepness, surface, tollways
</ul>
</div>
</div>

<div style="margin:0px 20px; width: 50%; display: inline-block;vertical-align: top">
<h3>Functionalities</h3>
<ul style="line-height: 30px">
    <li>`bearing`, `radiuses`: Control input point snapping behavior
    <li>`continue_straight`: Avoid u-turns, even if faster
    <li>`avoid_*`: Avoid features, e.g. ferry, **countries**, **polygons**
    <li>vehicle dimensions: use OSM dimension tags to route compliantly
    <li>**green**/**quiet**: prefer green and/or quiet routes (**only** available in Germany right now)
    <li>**landmark** backed pedestrian navigation (**only** available in Germany)
</ul>
</div>

## Directions - Avoid features for `cycling-regular`

In [ ]:
# Defines widget, for complete list see http://ipywidgets.readthedocs.io/en/latest/examples/Widget%20List.html
widget_hills = widgets.ToggleButtons(
                    options=['hills', 'unpavedroads', 'none'],
                    description='Avoid:',
                    disabled=False,
                    button_style='info', # 'success', 'info', 'warning', 'danger' or ''
                    )

# Event callback for on_change event of widget. 'change' is the emittant with 4 dict entries. 
# We're interested in 'new': new value of widget
def on_change_avoid(change):    
    # Call the routing function and parse
    route = avoidHills(change['new'])
    bbox = route['bbox']
    bbox_centroid = (average([bbox[1],bbox[3]]),average([bbox[0],bbox[2]]))
    route_properties = route['features'][0]['properties']
    
    # Clear all output, otherwise maps are appended to the figure
    clear_output()
    
    # Set up the folium map with custom HTML popup
    fig1, map1 = createBaseMap('100%', '400', map_provider='OpenStreetMap', coords=bbox_centroid, zoom=12)
    popup_html = """<strong>duration: </strong>{0:.1f} mins<br>
                    <strong>distance: </strong>{1:.3f} km""".format(route_properties['summary'][0]['duration'] / 60,
                                                           route_properties['summary'][0]['distance'] / 1000)
    
    iframe = branca.element.IFrame(html=popup_html, width=300, height=150)
    popup = folium.Popup(iframe, max_width=300)
    
    gj = folium.features.GeoJson(route, style_function=style_function())
    gj.add_child(popup)
    gj.add_to(map1)
    map1.add_to(fig1)
    
    # Displays widget in calling cell, i.e. below
    display(widget_hills)
    display(fig1)
    
# Widget event handler function        
widget_hills.observe(on_change_avoid, names='value')

In [61]:
import openrouteservice as ors

ors_key = api_key["ORS"] # keys are stored in local file
clnt = ors.Client(key=ors_key) # set up client
# This function is called by the widget's event callback
def avoidHills(mode):
    params = {'coordinates': [[8.681602, 49.422141],[8.737221, 49.49333]],
              'profile': 'cycling-regular',
              'format_out': 'geojson'}
    if mode != 'none':
        params['options'] = {'avoid_features': mode}
    route = clnt.directions(**params)
    return route

widget_hills

ToggleButtons(button_style='info', description='Avoid:', index=2, options=('hills', 'unpavedroads', 'none'), v…

## Directions - Avoid countries for `driving-hgv`

In [ ]:
widget_country = widgets.SelectMultiple(
    options={'DE':75, 'CH': 193, 'AT': 11, 'None': 'None'},
    value=[75],
    rows=4,
    description='Countries',
    disabled=False
)

def on_change_country(change):
    route = avoidCountry(change['new'])
    bbox = route['bbox']
    bbox_centroid = (average([bbox[1],bbox[3]]),average([bbox[0],bbox[2]]))
    route_properties = route['features'][0]['properties']
    
    # Clear all output
    clear_output()
    
    fig1, map1 = createBaseMap('100%', '400', map_provider='OpenStreetMap', coords=bbox_centroid, zoom=6)
    popup_html = """<strong>duration: </strong>{0:.1f} mins<br>
                    <strong>distance: </strong>{1:.3f} km""".format(route_properties['summary'][0]['duration'] / 60,
                                                           route_properties['summary'][0]['distance'] / 1000)
    
    iframe = branca.element.IFrame(html=popup_html, width=300, height=150)
    popup = folium.Popup(iframe, max_width=300)
    
    gj = folium.features.GeoJson(route, style_function=style_function())
    gj.add_child(popup)
    gj.add_to(map1)
    map1.add_to(fig1)
    
    display(widget_country)
    display(fig1)
    
        
widget_country.observe(on_change_country, names='value')

In [62]:
import openrouteservice as ors

ors_key = api_key["ORS"] # comes from local file
clnt = ors.Client(key=ors_key) # set up client
# This function is called by the widget's event callback
def avoidCountry(country):
    country = '|'.join([str(c) for c in country])
    params = {'coordinates': [[8.706665, 49.50381],[9.217529, 45.521744]],
              'profile': 'driving-hgv',
              'format_out': 'geojson',}
    if country != 'None':
        params['options'] = {'avoid_countries': country}
    route = clnt.directions(**params)
    return route

widget_country

SelectMultiple(description='Countries', index=(0,), options={'DE': 75, 'CH': 193, 'AT': 11, 'None': 'None'}, r…

## Directions - Route green or quiet for `foot-walking`

In [ ]:
widget_green = widgets.ToggleButtons(
                    options=['green', 'quiet', 'none'],
                    description='Preference:',
                    disabled=False,
                    button_style='info', # 'success', 'info', 'warning', 'danger' or ''
                    )

def on_change_green(change):    
    route = routeGreen(change['new'])
    bbox = route['bbox']
    bbox_centroid = (average([bbox[1],bbox[3]]),average([bbox[0],bbox[2]]))
    route_properties = route['features'][0]['properties']
    
    # Clear all output
    clear_output()
    
    fig1, map1 = createBaseMap('100%', '400', map_provider='OpenStreetMap', coords=bbox_centroid, zoom=13)
    popup_html = """<strong>duration: </strong>{0:.1f} mins<br>
                    <strong>distance: </strong>{1:.3f} km""".format(route_properties['summary'][0]['duration'] / 60,
                                                           route_properties['summary'][0]['distance'] / 1000)
    
    iframe = branca.element.IFrame(html=popup_html, width=300, height=150)
    popup = folium.Popup(iframe, max_width=300)
    
    gj = folium.features.GeoJson(route, style_function=style_function())
    gj.add_child(popup)
    gj.add_to(map1)
    map1.add_to(fig1)
    
    display(widget_green)
    display(fig1)
    
        
widget_green.observe(on_change_green, names='value')

In [63]:
import openrouteservice as ors

ors_key = api_key["ORS"] # comes from local file
clnt = ors.Client(key=ors_key) # set up client
# This function is called by the widget's event callback
def routeGreen(pref):
    params = {'coordinates': [[8.690872, 49.406673],[8.692117, 49.430124]],
              'profile': 'foot-walking',
              'optimized': 'false',
              'format_out': 'geojson'}
    if pref != 'none':
        params['options'] = {'profile_params': {'weightings': {pref: {'factor': 1.0}}}}
    route = clnt.directions(**params)
    return route

widget_green

ToggleButtons(button_style='info', description='Preference:', options=('green', 'quiet', 'none'), value='green…

## Isochrones API

<div style="margin:0px 20px; width: 40%; display: inline-block; vertical-align: top;">
<h3>Features</h3>
<br>
<div style=" border-right: 1px solid black">
<ul>
    <li>9 routing profiles:
        <ul>
            <li>2 car: `driving-car`, `driving-hgv`
            <li>5 cycling, incl. `*-mountain`, `*-safe`
            <li>2 walking: `foot-walking`, `foot-hiking`
        </ul>
    <li>Query up to 5 locations and 10 intervals in one request
</ul>
</div>
</div>

<div style="margin:0px 20px; width: 50%; display: inline-block;vertical-align: top">
<h3>Functionalities</h3>
<ul style="line-height: 30px">
    <li>**`attributes`**: Returns e.g. total population within isochrone<sup>1</sup>
    <li>plus similar functionalities as **Directions** API
</ul>
</div>

<br>
<br>

<small><sup>1</sup> Global data coverage by [EU GSH](https://ghsl.jrc.ec.europa.eu/about.php)</small>

## Isochrones - Retrieve population

In [ ]:
widget_iso = widgets.ToggleButtons(
                    options=['time', 'distance'],
                    description='Type: ',
                    disabled=False,
                    button_style='info', # 'success', 'info', 'warning', 'danger' or ''
                    )

def on_change_iso(change):    
    features = calculateIso(change['new'])
    bbox = features['bbox']
    bbox_centroid = (average([bbox[1],bbox[3]]),average([bbox[0],bbox[2]]))
    
    # Clear all output
    clear_output()
    
    fig1, map1 = createBaseMap('100%', '400', map_provider='OpenStreetMap', coords=bbox_centroid, zoom=5)
    
    for iso in features['features']:
        route_properties = iso['properties']
    
        unit = '30 mins' if features['info']['query']['range_type'] == 'time' else '50 km'
        popup_html = "{} people living within {} driving time.".format(route_properties['total_pop'], unit)
    
        iframe = branca.element.IFrame(html=popup_html, width=300, height=150)
        popup = folium.Popup(iframe, max_width=300)

        gj = folium.features.GeoJson(iso, style_function=style_function(0.5))
        gj.add_child(popup)
        gj.add_to(map1)
        
    map1.add_to(fig1)
    
    display(widget_iso)
    display(fig1)
    
        
widget_iso.observe(on_change_iso, names='value')

In [64]:
import openrouteservice as ors

ors_key = api_key["ORS"] # comes from local file
clnt = ors.Client(key=ors_key) # set up client
# This function is called by the widget's event callback
def calculateIso(dim):
    number = 1800 if dim == 'time' else 50000
    # Milan, Heidelberg, Berlin
    params = {'locations': [[9.170837, 45.47554],[8.684692, 49.409186], [13.41332, 52.521626]],
              'profile': 'driving-car',
              'range_type': dim,
              'intervals': [number],
              'segments': number,
              'attributes': ['total_pop']}
    iso = clnt.isochrones(**params)
    return iso

widget_iso

ToggleButtons(button_style='info', description='Type: ', index=1, options=('time', 'distance'), value='distanc…

## Geocoding API

Fork of **[Pelias](https://github.com/pelias/pelias)** geocoding engine.

<div style="margin:0px 20px; width: 40%; display: inline-block; vertical-align: top;">
<h3>Features</h3>
<br>
<div style=" border-right: 1px solid black">
<ul>
    <li>3 endpoints:
        <ul>
            <li>`geocoding/search?`
            <li>`geocoding/reverse?`
            <li>`geocoding/autocomplete?`
        </ul>
    <li>4 data sources:
        <ul>
            <li>**WhosOnFirst**: Global administrative boundaries
            <li>**OSM**
            <li>**openaddresses**: ~ 500 Mio addresses
            <li>**geonames**: Additional place names and locations
</ul>
</div>
</div>

<div style="margin:0px 20px; width: 50%; display: inline-block;vertical-align: top">
<h3>Functionalities</h3>
<ul style="line-height: 30px">
    <li>**libpostal**: address parser and normalizer
    <li>**placeholder**: address parser for unstructured text
    <li>**interpolation**: attempts interpolation for unknown house numbers
</ul>
<br>
<br>
These extra services are stand-alone API's and can be used in any application.
</div>

## Geocoding - Running out of names?!

In [ ]:
widget_geocode = widgets.ToggleButtons(
                    options=['Heidelberg', 'Milan', 'Springfield'],
                    description='Type: ',
                    disabled=False,
                    button_style='info', # 'success', 'info', 'warning', 'danger' or ''
                    )

def on_change_geocode(change):    
    features = geocode(change['new'])
    bbox = features['bbox']
    bbox_centroid = (average([bbox[1],bbox[3]]),average([bbox[0],bbox[2]]))
    us_cities = len([place for place in features['features'] if place['properties']['country'] == 'United States'])
    
    # Clear all output
    clear_output()
    
    fig1, map1 = createBaseMap('100%', '400', map_provider='OpenStreetMap', coords=bbox_centroid, zoom=2)
    print("Amount of cities worldwide:\t{}\nAmount of cities in US:  \t{}".format(len(features['features']), us_cities))
    
    for point in features['features']:
        route_properties = point['properties']
    
        popup_html = "{}<br>{}<br>{}".format(route_properties.get('name', ''), 
                                           route_properties.get('macroregion', ''),
                                           route_properties.get('country', '')
                                          )
    
        iframe = branca.element.IFrame(html=popup_html, width=300, height=150)
        popup = folium.Popup(iframe, max_width=300)

        gj = folium.features.GeoJson(point)
        gj.add_child(popup)
        gj.add_to(map1)
        
    map1.add_to(fig1)
    
    display(widget_geocode)
    display(fig1)
    
widget_geocode.observe(on_change_geocode, names='value')

In [65]:
import openrouteservice as ors

ors_key = api_key["ORS"] # comes from local file
clnt = ors.Client(key=ors_key) # set up client
# This function is called by the widget's event callback
def geocode(name):
    route = clnt.pelias_search(name, size=50)
    return route

widget_geocode

Amount of cities worldwide:	27
Amount of cities in US:  	20


ToggleButtons(button_style='info', description='Type: ', index=1, options=('Heidelberg', 'Milan', 'Springfield…

<div style="width: 48%; display: inline-block; vertical-align: top;">
<h2>Matrix API</h2>
<br>
    <h3>Features</h3>
    <ul>
        <li>Calculates OD matrices for up to **50x50 locations**
        <li>For all 9 profiles
        <li>Both `distance` and `duration` are returned
    </ul>
</div>

<div style="padding-left: 40px; border-left: 1px solid black; width: 48%; display: inline-block; vertical-align: top;">
<h2>POI API</h2>
<br>
    <div style="">
        <h3>Features</h3>
        <ul>
            <li>Query OSM POI's for up to 5 categories in one request
            <li>Use GeoJSON geometries or `bbox` to filter geographically
            <li>Use OSM tags to filter POI's, e.g. `smoking: no`, `wheelchair: yes`
            <li>Returns up to 2000 POI's per request

        </ul>
    </div>
</div>

## POIs - What's around us

In [ ]:
def populate_list(pois):
    cat_list = []
    for feature in pois['features']:
        categories = feature['properties']['category_ids']
        for category in categories:
            cat_list.append(categories[category]['category_name'])
    return sorted(set(cat_list))
    
milan = [9.226863, 45.479302]
def on_change_pois(change):    
    features = getPois(milan)
    print(features)
    # Clear all output
    clear_output()
    
    fig1, map1 = createBaseMap('100%', '400', map_provider='OpenStreetMap', coords=list(reversed(milan)), zoom=14)
    
    for point in features['features']:
        categories = point['properties']['category_ids']
        for category in categories:
            if categories[category]['category_name'] == change['new']:
                
                gj = folium.features.GeoJson(point)
                gj.add_to(map1)
        
    map1.add_to(fig1)
    
    display(widget_pois)
    display(fig1)

In [67]:
import openrouteservice as ors
from geojson import Point

ors_key = api_key["ORS"] # comes from local file
clnt = ors.Client(key=ors_key) # set up client
# This function is called by the widget's event callback
def getPois(location):
    params = {'request': 'pois',
            'geojson': Point(location),
            'buffer': 1000, # 1km radius around SOTM location
#             'filter_category_ids': [570],
            'sortby': 'distance'}
    route = clnt.places(**params)
    return route
pois = getPois(milan)
options = populate_list(pois)
widget_pois = widgets.Dropdown(options=options)
widget_pois.observe(on_change_pois, names='value')
widget_pois

Dropdown(index=88, options=('adult_gaming_centre', 'artwork', 'atm', 'bakery', 'bank', 'bar', 'beauty', 'bench…

## Infrastructure

<div style="display: inline-block; text-align:left; width: 36%; vertical-align: top; ">
    <h3>Clients</h3>
    <br>
    <div style="margin:0px 20px; width: 30%; display: inline-block; vertical-align: top">
        <img src="../img/python.png" style="height:150px">
    </div>
    <div style="display: inline-block; vertical-align: top;">
        **Python** client
        <br>
        <div style="padding: 5px 20px; display: inline-block; vertical-align: middle;">
            <figure>
                <a href="https://github.com/GIScience/openrouteservice-py"><img src="../img/github.png" style="height:2em"></a>
                <figcaption><center><small>Github</small></center></figcaption>
            </figure>
        </div>
        <div style="padding: 5px 20px; display: inline-block; vertical-align: middle;">
            <figure>
                <a href="https://pypi.org/project/openrouteservice/"><img src="../img/pypi.png" style="height:2em"></a>
                <figcaption><center><small>PyPI</small></center></figcaption>
            </figure>
        </div>
        <div style="padding: 5px 20px; display: inline-block; vertical-align: middle;">
            <figure>
                <a href="https://openrouteservice-py.readthedocs.io/en/latest/"><img src="../img/rtd.png" style="height:2em"></a>
                <figcaption><center><small>RTD</small></center></figcaption>
            </figure>
        </div>
    </div>   
    <hr style="margin-right: 20px">

    <div style="margin:0px 20px; width: 30%; display: inline-block; vertical-align: top">
        <img src="../img/r.png" style="height:150px">
    </div>
    <div style="display: inline-block; vertical-align: top;">
        **R** client
        <br>
        <div style="padding: 5px 20px; display: inline-block; vertical-align: middle;">
            <figure>
                <a href="https://github.com/GIScience/openrouteservice-r"><img src="../img/github.png" style="height:2em"></a>
                <figcaption><center><small>Github</small></center></figcaption>
            </figure>
        </div>
        <div style="padding: 5px 20px; display: inline-block; vertical-align: middle;">
            <figure>
                <a href="https://giscience.github.io/openrouteservice-r/articles/openrouteservice.html"><img src="../img/documentation.png" style="height:2em"></a>
                <figcaption><center><small>Docu</small></center></figcaption>
            </figure>
        </div>
    </div>   
    <hr style="margin-right: 20px">

    <div style="margin:0px 20px; width: 30%; display: inline-block; vertical-align: top">
        <img src="../img/qgis.png" style="height:150px">
    </div>
    <div style="display: inline-block; vertical-align: top;">
        **QGIS** plugin
        <br>
        <div style="padding: 5px 20px; display: inline-block; vertical-align: middle;">
            <figure>
                <a href="https://github.com/nilsnolde/OSMtools"><img src="../img/github.png" style="height:2em"></a>
                <figcaption><center><small>Github</small></center></figcaption>
            </figure>
        </div>
        <div style="padding: 5px 20px; display: inline-block; vertical-align: middle;">
            <figure>
                <a href="https://plugins.qgis.org/plugins/OSMtools/"><img src="../img/download.png" style="height:2em"></a>
                <figcaption><center><small>Plugin repo</small></center></figcaption>
            </figure>
        </div>
    </div>   
    <hr style="margin-right: 20px">

    <div style="margin:0px 20px; width: 30%; display: inline-block; vertical-align: top">
        <img src="../img/app.png" style="height:150px">
    </div>
    <div style="display: inline-block; vertical-align: top;">
        Web app
        <br>
        <div style="padding: 5px 10px; display: inline-block; vertical-align: middle;">
            <figure>
                <a href="https://github.com/GIScience/openrouteservice-app"><img src="../img/github.png" style="height:2em"></a>
                <figcaption><center><small>Github</small></center></figcaption>
            </figure>
        </div>
    </div>   
    <hr style="margin-right: 20px">
</div>

<div style="padding-left: 40px; display: inline-block; text-align:left; width: 60%; vertical-align: top; border-left: 1px solid black">
    <h3>Microservices</h3>
    <br>
    <div style="margin:0px 20px; width: 30%; display: inline-block; vertical-align: top">
        <figure>
            <img src="../img/places.png" style="height:150px">
            <figcaption><strong>openpoiservice</strong></figcaption>
        </figure>
    </div>
    <div style="width: 60%; display: inline-block; vertical-align: top;">
        Flask application to serve OSM Places of Interest (POI's) via a Rest API.
        <h5>Features</h5>
        <ul>
            <li>fully self-contained microservice to create, drop and populate PostGIS tables
            <li>`POST` endpoint, which accepts GeoJSON as geometry and OSM tags as filters
            <li>Customizable setup with Docker container
        </ul>
        <br>
        <div style="padding: 5px 10px; display: inline-block; vertical-align: middle;">
            <figure style="padding: 10px 0">
                <a href="https://github.com/GIScience/openpoiservice"><img src="../img/github.png" style="height:2em"></a>
                <figcaption><small>Github</small></figcaption>
            </figure>
        </div>
    </div>
    <h3>_Coming Soon!!_</h3>
    <div style="margin:0px 20px; width: 30%; display: inline-block; vertical-align: top">
        <figure>
            <img src="../img/openfuelservice.png" style="height:150px">
            <figcaption><strong>openfuelservice</strong></figcaption>
        </figure>
    </div>
    <div style="width: 60%; display: inline-block; vertical-align: top;">
        Flask application to estimate fuel consumption on > 600 car models. Takes GeoJSON and car model as input.
    </div>
</div>

# You know what's on for dinner?

Let's see how OSM and Yelp make the restaurant choice a little easier:

- Find restaurants within 10 min walking distance from SOTM location
- Filter restaurants with Yelp rating >= 4
- Create widget to filter by restaurant category

## Set up *openrouteservice* client

1. Sign up on our [homepage](https://openrouteservice.org/sign-up/) 
2. Create API key on our [developer portal](https://openrouteservice.org/dev/#/home)
3. Review our [ToS](https://openrouteservice.org/terms-of-service/)
4. `pip install openrouteservice`
5. Fire away!

Here we:
- _**Geocode**_ the SOTM official address
- Create an _**isochrone**_ with 10 mins walking radius

In [74]:
import openrouteservice as ors

clnt = ors.Client(key='5b3ce3597851110001cf624870cf2f2a58d44c718542b3088221b684')

params_geocode = {'text': 'Politecnico di Milano ‐ Piazza Leonardo da Vinci, Milan, Lombardy, Italy'}

sotm_location = clnt.pelias_search(**params_geocode)['features'] # Geocode from official address
sotm_isochrone = clnt.isochrones(locations=sotm_location[0]['geometry']['coordinates'], # Calculate isochrones
                                 intervals=[600],
                                 segments=600,
                                 profile='foot-walking')

In [75]:
x, y = sotm_location[0]['geometry']['coordinates']
def createWidgetBaseMap(width, height, include=True, map_provider='stamenwatercolor'):
    bar_icon = folium.features.CustomIcon('https://openrouteservice.org/wp-content/uploads/2017/07/bar.png',
                                       icon_size=(30, 30))
    sotm_icon = folium.features.CustomIcon('https://openrouteservice.org/wp-content/uploads/2017/07/sotm_map_marker.png', icon_size=(58, 70))

    fig = folium.Figure(width=width, height=height)
    m = folium.Map(tiles=map_provider,location=(y, x), zoom_start=15)
    folium.map.Marker([y, x], icon=sotm_icon).add_to(m)
    if include:
        folium.features.GeoJson(sotm_isochrone).add_to(m)
    
    m.add_child(MeasureControl())
    
    return fig, m

## Set up a basic `folium` map

- The `createWidgetBaseMap` factory sets up the initial map, so we start fresh every time

In [76]:
fig, m = createWidgetBaseMap('100%', 600) # Call map factory with figure dimensions
fig.add_child(m)
fig

## Request restaurant POI's

- `/pois?` endpoint takes GeoJSON
- we can pipe the GeoJSON output of `/isochrones?` directly to `/pois?` to limit the query to the isochrones geometry

In [77]:
param_pois = {'request': 'pois',
            'geojson': sotm_isochrone['features'][0]['geometry'],
            'filter_category_ids': [570], # ID list: https://github.com/GIScience/openrouteservice-docs#sustenance--560
            'sortby': 'distance'}

sotm_restaurants = clnt.places(**param_pois)['features']
display(HTML('<br><br>Amount of restaurants within 10 mins walking radius: <strong>{}</strong>'.format(len(sotm_restaurants))))

## Yelp rating

- use the [Yelp businesses API](https://www.yelp.de/developers/documentation/v3/business) to query the reviews of our restaurants
- extract `name`, `rating` and `url` to feature popup

In [78]:
yelp_url = 'https://api.yelp.com/v3/businesses/search?' # Create a Yelp account and a API key: https://www.yelp.com/developers
yelp_header = {'Authorization': api_key['Authorization']} # use secret from file: {'Authorization': 'Bearer MyAPIkey'}
yelp_params = {'limit': 1}

def yelp_response(**kwargs):
    yelp_params.update({'term': r_name,
                        'longitude': r_coords[0],
                        'latitude': r_coords[1]})
    # Concat params to URL encoded string
    yelp_url_params = requests.utils.unquote_unreserved(urlencode(list(yelp_params.items())))
    try:
        yelp_response = requests.get(yelp_url + yelp_url_params,
                                     headers=yelp_header).json()['businesses'][0]
    except:
        return False

    if yelp_response['rating'] < 4.0:
        return False
    
    return yelp_response

In [79]:
import branca

fig, m = createWidgetBaseMap('100%', '500')
for restaurant in sotm_restaurants:
    r_name = restaurant['properties']['osm_tags'].get('name', '')
    if r_name != '':
        r_coords = restaurant['geometry']['coordinates']
        yelp_return = yelp_response(r_name=r_name, # Call hidden function to retrieve Yelp results with rating >= 4 stars
                                    r_coords=r_coords)
        if yelp_return:
            restaurant['yelp'] = yelp_return
            restaurant_icon = folium.features.CustomIcon('https://openrouteservice.org/wp-content/uploads/2017/07/restaurant2.png',
                                                   icon_size=(50, 50))
            popup_html = '<h4>{0}</h4><strong><a href="{1}" target="_blank">URL</a><br>Stars: </strong>{2}'.format(r_name, yelp_return['url'], yelp_return['rating'])
            iframe = branca.element.IFrame(html=popup_html, width=300, height=100)
            popup = folium.Popup(iframe, max_width=300)
            folium.map.Marker(list(reversed(r_coords)), icon=restaurant_icon, popup=popup).add_to(m)
fig.add_child(m)
fig

In [82]:

    # Milan, Heidelberg, Berlin# Define change function for drop-down
def on_category_change(change):
    # First we need to clear_output() entirely and then re-draw all components
    clear_output()
    
    x, y = sotm_location[0]['geometry']['coordinates']
    fig2, map2 = createWidgetBaseMap('100%', 600, include=False)
    
    for restaurant in sotm_restaurants:
        if restaurant.get('yelp', '') != '':
            if restaurant['yelp']['categories'][0]['title'] == change['new']:
                restaurant_coords = restaurant['geometry']['coordinates']
                clnt = ors.Client(key='5b3ce3597851110001cf624870cf2f2a58d44c718542b3088221b684')
                route = clnt.directions(coordinates=[[x,y],
                                                     restaurant_coords
                                                    ],
                                       format_out='geojson',
                                       profile='foot-walking',
                                       geometry_format='geojson')
                
                route_properties = route['features'][0]['properties']
                folium.features.GeoJson(route).add_to(map2)

                restaurant_icon = folium.features.CustomIcon('https://openrouteservice.org/wp-content/uploads/2017/07/restaurant2.png',
                                                             icon_size=(50, 50))
                popup_html = """<h4>{0}</h4>
                                <strong><a href="{1}" target="_blank">URL</a></strong><br>
                                <strong>Category: </strong>{2}<br>
                                <strong>Walking time: </strong>{3:.2f} mins<br>
                                <strong>Reviews: </strong>{4}<br>                                
                                <strong>Stars: </strong>{5}""".format(restaurant['yelp']['name'],
                                                              restaurant['yelp']['url'],
                                                              change['new'],
                                                              route_properties['summary'][0]['duration'] / 60,
                                                              restaurant['yelp']['review_count'],
                                                              restaurant['yelp']['rating'])
                
                iframe = branca.element.IFrame(html=popup_html, width=300, height=150)
                popup = folium.Popup(iframe, max_width=300)
                folium.map.Marker(list(reversed(restaurant_coords)),
                                  icon=restaurant_icon,
                                  popup=popup,
                                 ).add_to(map2)
    fig2.add_child(map2)
    # Now we re-add the widgets to the cleared output area
    display(widget_dd)
    display(fig2)

## IPython widget

The final product is a drop-down widget to query the directions to nearby restaurants by restaurant category:

In [83]:
category_titles = []
for restaurant in sotm_restaurants:
    if restaurant.get('yelp', '') != '':
        category_titles.append(restaurant['yelp']['categories'][0]['title'])

widget_dd = widgets.Dropdown(options=set(category_titles)) # Create widget with categories in drop-down
widget_dd.observe(on_category_change, names='value') # Hidden method is called by the listener        
display(widget_dd)

Dropdown(index=9, options=('Asian Fusion', 'Lumbard', 'African', 'Lounges', 'Bistros', 'Convenience Stores', '…

## Thanks to...

- **Pelias** and **GraphHopper** crews for developing cutting-edge open-source tech!
- **Damian Avila** for the `rise` development which powers this presentation
- **Jupyter** development crew
- **Yelp** for providing a free attractions API

The source code and PDF of this presentation can be found [here](https://github.com/nilsnolde/sotm2018).

<center><h1>Thanks!!</h1></center>
<div style="width: 48%; display: inline-block; vertical-align: top;">
<br>
    <h3>Features</h3>
    <ul>
        <li>Calculates OD matrices for up to **50x50 locations**
        <li>For all 9 profiles
        <li>Both `distance` and `duration` are returned
    </ul>
</div>

<div style="padding-left: 40px; border-left: 1px solid black; width: 48%; display: inline-block; vertical-align: top;">
<h2>POI API</h2>
<br>
    <div style="">
        <h3>Features</h3>
        <ul>
            <li>Query OSM POI's for up to 5 categories in one request
            <li>Use GeoJSON geometries or `bbox` to filter geographically
            <li>Use OSM tags to filter POI's, e.g. `smoking: no`, `wheelchair: yes`
            <li>Returns up to 2000 POI's per request

        </ul>
    </div>
</div>